In [8]:
from MFEA_lib.model import AbstractModel, SMP_MFEA, MFEA_base, SMP_DE
from MFEA_lib.model.utils import *
from MFEA_lib.model.utils import TuningModel
import pandas as pd
import numpy as np
import scipy.io
from MFEA_lib.tasks.Benchmark.Funcs import * 
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *
from MFEA_lib.operators.Search import *

### Compare Benchmark

In [9]:
compare = CompareResultBenchmark(path_folder="./RESULTS/result/GECCO20", ls_benchmark=[WCCI22_benchmark.get_50tasks_benchmark(i)[0] for i in range(1, 11)])

(0 : LSA_2021)
(1 : SMP_V2)
(2 : LSA_21_NOLOCAL)
(3 : SBS_GA)
(4 : LSA)
(5 : SA)


In [12]:
compare.summarizing_compare_result(idx_main_algo= 1,min_value=1e-6)

['LSA_2021', 'SMP_V2', 'LSA_21_NOLOCAL', 'SBS_GA', 'LSA', 'SA']


,Better,Equal,Worse
SMP_V2 vs LSA_2021,97,67,336
SMP_V2 vs LSA_21_NOLOCAL,34,137,329
SMP_V2 vs SBS_GA,387,0,113
SMP_V2 vs LSA,211,73,216
SMP_V2 vs SA,293,67,140


In [13]:
compare.show_compare_detail(min_value=1e-6, round=7,idx_main_algo=1)

Benchmark:  1
         SMP_V2 LSA_2021 LSA_21_NOLOCAL    SBS_GA        LSA      SA
Tasks1   (+)0.0   (+)0.0         (+)0.0   1.9e-05     (+)0.0  (+)0.0
Tasks2   (+)0.0   (+)0.0         (+)0.0  1.91e-05    2.6e-06  (+)0.0
Tasks3   (+)0.0   (+)0.0         (+)0.0  1.73e-05     (+)0.0  (+)0.0
Tasks4   (+)0.0   (+)0.0         (+)0.0   1.7e-05     (+)0.0  (+)0.0
Tasks5   (+)0.0   (+)0.0         (+)0.0  1.66e-05     (+)0.0  (+)0.0
Tasks6   (+)0.0   (+)0.0         (+)0.0  2.01e-05     (+)0.0  (+)0.0
Tasks7   (+)0.0   (+)0.0         (+)0.0  1.95e-05     (+)0.0  (+)0.0
Tasks8   (+)0.0   (+)0.0         (+)0.0  2.08e-05     (+)0.0  (+)0.0
Tasks9   (+)0.0   (+)0.0         (+)0.0  2.29e-05     (+)0.0  (+)0.0
Tasks10  (+)0.0   (+)0.0         (+)0.0  1.82e-05     (+)0.0  (+)0.0
Tasks11  (+)0.0   (+)0.0         (+)0.0  1.89e-05     (+)0.0  (+)0.0
Tasks12  (+)0.0   (+)0.0         (+)0.0  1.64e-05     (+)0.0  (+)0.0
Tasks13  (+)0.0   (+)0.0         (+)0.0  1.77e-05     (+)0.0  (+)0.0
Tasks14  (+)0.0   (+

### Tuning Model

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
name_benchmarks = []
for id in range(1, 10):
    # print(id) 
    b, i = WCCI22_benchmark.get_complex_benchmark(4)
    name_benchmarks.append("wcci_2tasks_" + str(id))
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)
name_benchmarks.append("CEC_10_")

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 3): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

In [ ]:
model = TuningModel(
    model_name= SMP_MFEA, 
    list_parameter= [
        ('crossover', {
            'gamma': [0.4, 0.6]
        }), 
        ('lr', [0.04, 0.05]),
        ('crossover', {
            'alpha': [5, 7]
        }), 
    ],
)
model.compile(
    ls_benchmark = ls_benchmark, 
    benchmark_weights = (np.ones(shape= (3,)) / 3).tolist(), 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,

    # tasks = None,
    # IndClass= None,
    # tasks = CEC17_benchmark.get_10tasks_benchmark(),
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    # crossover= SBX_Crossover(nc = 2),
    mutation = GaussMutation(scale = 0.01),
    selection= ElitismSelection(random_percent= 0.),
    # search= SHADE(len_mem= 30, p_ontop= 0.1), 
    attr_tasks = ['crossover', 'mutation']
)

fit, com, result = model.run(
    replace_folder=True,
    min_value = 1e-6,
    
    nb_generations= 5, 
    nb_inds_each_task= 100, 
    nb_inds_min= 100,
    lr = 0.05,  
    p_const_intra= 0, 
    p_mutate= 0.1, 
    prob_search = 0.,
    nb_epochs_stop= 1000, 
    swap_po= False,
    evaluate_initial_skillFactor= True
)

### Show and compare result

In [ ]:
for id in range(1, 11): 
    SMP = loadModel("./RESULTS/__static__/GECCO20/SMP_NEWSBX_TUN_50/Gecco_id_"+ str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    SA = loadModel("./RESULTS/__static__/GECCO20/SA/SA_Benchmark_"+str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    LSA_21 = loadModel("./RESULTS/__static__/GECCO20/LSA_2021/LSA_2021_MTOMSO_Benchmark_"+str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    com = CompareModel([SMP, SA, LSA_21])
    print(com.detail_compare_result(min_value= 1e-6, round = 7))

In [ ]:
MFEA_model = loadModel('./RESULTS/MFEA_cec17.mso')
EBS_GA_model = loadModel('./RESULTS/EBS_GA_cec17.mso')
MaTDE_model = loadModel('./RESULTS/MaTDE_cec17.mso')
MaTGA_model = loadModel('./RESULTS/MaTGA_cec17.mso')
MTEA_AD_model = loadModel('./RESULTS/MTEA_AD_cec17.mso')
SA_model = loadModel('./RESULTS/SA_cec17.mso')
LSA_model = loadModel('./RESULTS/LSA_cec17.mso')
# SMP_NEWSBX = loadModel("./RESULTS/SMP_NEWSBX.mso", ls_tasks= CEC17_benchmark.get_10tasks_benchmark())


In [ ]:
compare = CompareModel(
    models = [MFEA_model, EBS_GA_model, MaTDE_model, MaTGA_model, MTEA_AD_model, SA_model, LSA_model],
    label= [...] * 7 ,
)
# compare.render(    
#     shape = (2, 5),
#     min_cost= 0,
#     step= 100,
#     yscale= 'log'
# )

In [ ]:
compare.detail_compare_result()

In [ ]:
compare.summarizing_compare_result(path=None , idx_main_algo= 2, nb_task= 2, ls_benchmark= [CEC17_benchmark.get_2tasks_benchmark(id) for id in range(1,10)])

In [ ]:
model = loadModel("./RESULTS/50tasks/SMP_NEWSBX/lan2_2s_newsbx_50task_1.mso")

In [ ]:
compare.summarizing_compare_result(path="./RESULTS/complex_2tasks/", idx_main_algo= 0, nb_task= 2, ls_benchmark= [WCCI22_benchmark.get_complex_benchmark(id)[0] for id in range(1,10)],min_value= 1e-6, combine= True)

In [ ]:
compare.summarizing_compare_result(path="./RESULTS/50tasks/", idx_main_algo= 7, nb_task= 50, ls_benchmark= [GECCO20_benchmark_50tasks.get_items(id) for id in range(1,11)],min_value= 1e-6, combine= False)

### Multi Time Model

In [ ]:
for id in range(4, 11): 
    tasks, IndClass= WCCI22_benchmark.get_50tasks_benchmark(id)

    smpModel = MultiTimeModel(model= SMP_MFEA)
    smpModel.compile(
        tasks= tasks,
        IndClass = IndClass,
        crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
        mutation= GaussMutation(scale= 0.1),
        # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
        selection= ElitismSelection(random_percent= 0.),
        attr_tasks = ['crossover', 'mutation'],
        
    )
    smpModel.fit(
        nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
        lr = 0.15, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
        nb_epochs_stop= 1500, 
        evaluate_initial_skillFactor= True
    )
    smpModel.run(
        nb_run= 1,
        save_path= './RESULTS/results/smpv2_50tasks_id_'+ str(id) + '.mso'
    )

In [ ]:
for id in range(1, 11): 
    tasks, IndClass= WCCI22_benchmark.get_50tasks_benchmark(id)

    smpModel = MultiTimeModel(model= SMP_MFEA)
    smpModel.compile(
        tasks= tasks,
        IndClass = IndClass,
        crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
        mutation= GaussMutation(scale= 0.1),
        # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
        selection= ElitismSelection(random_percent= 0.),
        attr_tasks = ['crossover', 'mutation'],
        
    )
    smpModel.fit(
        nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
        lr = 0.15, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
        nb_epochs_stop= 1500, 
        evaluate_initial_skillFactor= True
    )
    smpModel.run(
        nb_run= 5,
        save_path= './RESULTS/result/WCCI22_50tasks_id_'+ str(id) + '.mso'
    )

In [ ]:
tasks, IndClass= CEC17_benchmark.get_10tasks_benchmark()

smpModel = MultiTimeModel(model= SMP_MFEA)
smpModel.compile(
    tasks= tasks,
    IndClass = IndClass,
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.15, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 1500, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 30,
    save_path= './RESULTS/result/MFEA_cec17_final.mso'
)

In [ ]:
smpModel.print_result(['history_cost'])

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 10): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

smpModel = MultiBenchmark(
    ls_benchmark = ls_benchmark, 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,
    model= SMP_MFEA
    )

smpModel.compile(
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 10, nb_inds_each_task= 100, nb_inds_min= 100,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

In [ ]:
smpModel.print_result(['history_cost'])

### SMP DE

In [ ]:
tasks, IndClass= WCCI22_benchmark.get_50tasks_benchmark(8)

smpModel = MultiTimeModel(model= SMP_DE)
smpModel.compile(
    tasks= tasks,
    IndClass = IndClass,
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    search = SHADE(len_mem=30, p_ontop=0.1),
    attr_tasks = ['crossover', 'mutation', 'search'],
    
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 1,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 1,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

In [ ]:
lsa_21 = loadModel("./RESULTS/__static__/GECCO20/LSA_2021/LSA_2021_MTOMSO_Benchmark_8.mso",WCCI22_benchmark.get_50tasks_benchmark(8)[0])
model = loadModel('./RESULTS/result/smpMFEA_cec17.mso', WCCI22_benchmark.get_50tasks_benchmark(8)[0])

compare = CompareModel([lsa_21, model])
compare.detail_compare_result(min_value=1e-6)

### SMP - LSHADE

In [2]:
ls_benchmark = []
ls_IndClass = []

for i in range(3, 4, 1):
    t, ic = WCCI22_benchmark.get_complex_benchmark(i)
    ls_benchmark.append(t)
    ls_IndClass.append(ic)

name_benchmark = np.arange(len(ls_benchmark)) + 1

In [6]:
smpModel = MultiBenchmark(
    ls_benchmark= ls_benchmark,
    name_benchmark= name_benchmark,
    ls_IndClass= ls_IndClass,
    model= SMP_MFEA
)

smpModel.compile(
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    # mutation = GaussMutation(scale= 0.1),
    mutation = Directional_Mutation_v2(pm= 1/25),
    selection= ElitismSelection(random_percent= 0.2),
    search= L_SHADE(),
    attr_tasks = ['crossover', 'mutation'],
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.1, p_const_intra= 0.2, prob_search = 1.0,
    nb_epochs_stop= 1000, swap_po= False,
    evaluate_initial_skillFactor= True
)
a = smpModel.run(
    nb_run= 1,
    save_path= './RESULTS/result/WCCI22_complex/SMP_v2/'
)


Time: 02m 19.07s  100 % [====================>]  Pop_size: 3.00E+01  ,  Cost: 3.73E+06  1.71E+06  ,  
[0.5 0.5]
[100190.  99838.]
END!
DONE!
Saved


In [7]:
model = loadModel("./RESULTS/result/WCCI22_complex/SMP_v2/1.mso", ls_tasks=WCCI22_benchmark.get_complex_benchmark(1)[0])

In [ ]:
model.render_smp()

In [ ]:
compare = CompareResultBenchmark(
    path_folder= "./RESULTS/result/WCCI22_complex",
    ls_benchmark= [WCCI22_benchmark.get_complex_benchmark(i)[0] for i in range(1, 10)]
)

In [ ]:
compare.summarizing_compare_result()

In [ ]:
compare.show_compare_detail()

### Draft